In [209]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
from tqdm import tqdm
import itertools
from re import sub
from decimal import Decimal

In [210]:
pages=list(range(0,60, 10))

base_url='https://ca.indeed.com/salaries/'

positions=['Data-Scientist',
           'Data-Engineer',
           'Data-Analyst']

provinces=['Ontario',
          'Alberta',
          'British-Columbia',
          'Québec-Province',
          'Saskatchewan',
          'Yukon',
          'Nova-Scotia',
          'Manitoba',
          'New-Brunswick',
          'Newfoundland-And-Labrador',
          'Prince-Edward-Island',
          'Northwest-Territories',
          'Nunavut']

url_connect='-Salaries,-'
pre_df=[]
for position in tqdm(positions):
    for province in provinces:
        for page in pages:
            combined_url=base_url+position+url_connect+province+'?start='+str(page)
            response=requests.get(combined_url)
            soup=bs(response.text, 'lxml')
            results=soup.find_all('table', id="cmp-sal-company-table")
            for result in results:
                if result.findChild('caption'):
                    table_entries=result.find_all('tr', class_="cmp-salary-aggregate-table-entry")
                    for item in table_entries:
                        title=item.find('a').text.strip()
                        salary=item.find('div', class_="cmp-sal-summary").text.strip()
                        pre_df.append([province, position, title, salary, combined_url])
                else:
                    pass


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [01:43<00:00, 33.62s/it]


In [211]:
df=pd.DataFrame(pre_df)
df.columns=['Province', 'Position', 'Title', 'Salary per Year', 'url']

In [215]:
salaries=df['Salary per Year']
salaries_df=[]
for salary in salaries:
    salary, per, time =salary.split(' ')
    salaries_df.append([salary, time])

salaries_df=pd.DataFrame(salaries_df)
salaries_df.columns=['salary', 'per time']
salaries_df.head()
for i, row in salaries_df.iterrows():
    salary=salaries_df['salary'].iloc[i]
    value = Decimal(sub(r'[^\d.]', '', salary))
    salaries_df['salary'].iloc[i]=value

salaries_df['salary'].loc[salaries_df['per time'].str.contains('month')] = salaries_df['salary'] * 12
salaries_df['salary'].loc[salaries_df['per time'].str.contains('week')] = salaries_df['salary'] * 52
salaries_df['salary'].loc[salaries_df['per time'].str.contains('hour')] = salaries_df['salary'] * 2080
salaries_df
    
salaries_fixed=salaries_df['salary']

In [216]:
df['Salary per Year']=salaries_fixed
df

,Province,Position,Title,Salary per Year,url
0,Ontario,Data-Scientist,Quartic.ai Data Scientist,82508,https://ca.indeed.com/salaries/Data-Scientist-...
1,Ontario,Data-Scientist,Brainwerx Group Inc. Data Scientist,117211,https://ca.indeed.com/salaries/Data-Scientist-...
2,Ontario,Data-Scientist,Staffinity Inc. Data Scientist,103210,https://ca.indeed.com/salaries/Data-Scientist-...
3,Ontario,Data-Scientist,Shared Vision Management Search Data Scientist,102046,https://ca.indeed.com/salaries/Data-Scientist-...
4,Ontario,Data-Scientist,ProVision Data Scientist,101871,https://ca.indeed.com/salaries/Data-Scientist-...
5,Ontario,Data-Scientist,Michael Page International Data Scientist,89450,https://ca.indeed.com/salaries/Data-Scientist-...
6,Ontario,Data-Scientist,Rang Technologies Inc. Data Scientist,80000,https://ca.indeed.com/salaries/Data-Scientist-...
7,Ontario,Data-Scientist,CyberCoders Data Scientist,145001,https://ca.indeed.com/salaries/Data-Scientist-...
8,Ontario,Data-Scientist,Jobspring Partners Data Scientist,118740,https://ca.indeed.com/salaries/Data-Scientist-...
9,Ontario,Data-Scientist,Workbridge Associates Data Scientist,118160,https://ca.indeed.com/salaries/Data-Scientist-...
